In [11]:
import pandas as pd
import numpy as np
import math

import sqlite3
import json
import ast

In [12]:
lis_months = ["F","G","H","J","K","M","N","Q","U","V","X","Z"]
month_to_index = {m: i for i, m in enumerate(lis_months)}


In [13]:
def expand_contracts_and_lots(contracts, lots):
    if len(contracts) != len(lots):
        raise ValueError("contracts and lots must have same length")

    # ---- Parse contracts into (month_index, year) ----
    parsed = []
    for c in contracts:
        month = c[0]
        year = int(c[1:])  # last two digits
        parsed.append((month_to_index[month], year))

    # ---- Determine the start and end points ----
    start_month, start_year = parsed[0]
    end_month, end_year = parsed[-1]

    # ---- Expand continuously month-by-month ----
    expanded_contracts = []
    expanded_lots = []

    lot_map = {contracts[i]: lots[i] for i in range(len(contracts))}

    ym = start_month
    yy = start_year

    while True:
        # Make contract code
        contract_code = lis_months[ym] + str(yy)

        # append contract
        expanded_contracts.append(contract_code)

        # Append lot if exists, else 0
        expanded_lots.append(lot_map.get(contract_code, 0))

        # Stop condition
        if yy == end_year and ym == end_month:
            break

        # Increment month
        ym += 1
        if ym == 12:
            ym = 0
            yy += 1

    return expanded_contracts, expanded_lots

In [14]:
def next_contract(contract, step):
    month = contract[0]              # e.g. "H"
    year = int(contract[1:])         # e.g. "25"

    idx = month_to_index[month]         # find index of "H"
    new_idx = idx + step             # add step

    # adjust year when crossing December → January
    year += new_idx // 12            # integer rollover
    new_month = lis_months[new_idx % 12]

    return f"{new_month}{year}"

In [15]:
def contract_sort_key(contract):
    """
    Convert contract code like 'H25' → (year, month_index)
    so it can be sorted chronologically.
    """
    month = contract[0]
    year = int(contract[1:])  # handles 25, 2025, etc
    return (year, month_to_index[month])

In [16]:
contract_sort_key('H25')

(25, 2)

In [17]:
def create_custom_strategy_intraproduct(strategy_name, lis_lots):

    # ---- 1. Factorize lots ----
    non_zero_positive_sizes = [abs(x) for x in lis_lots if x != 0]

    if not non_zero_positive_sizes:
        raise ValueError("Lots list cannot have all zeros.")

    gcd = non_zero_positive_sizes[0]
    for n in non_zero_positive_sizes[1:]:
        gcd = math.gcd(gcd, n)

    lots_lis_factorized = [x // gcd for x in lis_lots]

    # ---- 2. Save to DB ----
    conn = sqlite3.connect("positionmanager.db")
    cur = conn.cursor()

    # Create table if not exists
    cur.execute("""
        CREATE TABLE IF NOT EXISTS custom_strategies (
            structure_name TEXT PRIMARY KEY,
            structure_lots TEXT
        )
    """)

    # Convert list → JSON for safe storage
    lots_json = json.dumps(lots_lis_factorized)

    # Insert or replace (overwrite if strategy name already exists)
    cur.execute("""
        INSERT OR REPLACE INTO custom_strategies (structure_name, structure_lots)
        VALUES (?, ?)
    """, (strategy_name, lots_json))

    conn.commit()
    conn.close()



In [23]:
import itertools

# --------- 1. Generate Outrights ---------
strategy_names = []
strategy_lots_list = []

# Outright = single contract = [1]
strategy_names.append("outright")
strategy_lots_list.append([1])

# --------- 2. Generate 1mo–12mo spreads ---------
for months in range(1, 13):
    name = f"{months}mo spread"
    pattern = [1] + [0]*(months-1) + [-1]   # Example: 3mo → [1,0,0,-1]
    
    strategy_names.append(name)
    strategy_lots_list.append(pattern)

# --------- 3. Generate 1mo–12mo butterflies (flies) ---------
for months in range(1, 13):
    name = f"{months}mo fly"
    
    # Length of pattern = 2*months + 1
    length = 2*months + 1
    
    # Example: 1mo fly → 3 long legs → length = 3
    # 2mo fly → length = 5 (1,0,-2,0,1)
    
    pattern = [0] * length
    pattern[0] = 1
    pattern[months] = -2
    pattern[-1] = 1
    
    strategy_names.append(name)
    strategy_lots_list.append(pattern)

# --------- 4. Populate DB using your function ---------
for name, lots in zip(strategy_names, strategy_lots_list):
    create_custom_strategy_intraproduct(name, lots)

print("DONE. All strategies inserted into DB.")

DONE. All strategies inserted into DB.


In [30]:
def hedge_outrights(lis_contracts, lis_lots):
    expanded_lis_contracts, expanded_lis_lots = expand_contracts_and_lots(lis_contracts, lis_lots)
    # ---- 1. Factorize lots ----
    non_zero_positive_sizes = [abs(x) for x in expanded_lis_lots if x != 0]

    if not non_zero_positive_sizes:
        raise ValueError("Lots list cannot have all zeros.")

    gcd = non_zero_positive_sizes[0]
    for n in non_zero_positive_sizes[1:]:
        gcd = math.gcd(gcd, n)

    lis_lots_factorized = [x // gcd for x in expanded_lis_lots]

# ---- 3. Fetch matching strategy from DB ----
    conn = sqlite3.connect("positionmanager.db")
    cursor = conn.cursor()

    cursor.execute("SELECT structure_name, structure_lots FROM custom_strategies")
    rows = cursor.fetchall()

    structure_name = None

    for name, lots_str in rows:
        stored_lots = ast.literal_eval(lots_str)   # safe parse "[1,-2,1]"
        if stored_lots == lis_lots_factorized:
            structure_name = name
            break

    conn.close()

    if structure_name is None:
        raise ValueError("No matching custom strategy found for factorized lots.")

    # ---- 4. Compute lots = lis_contracts[0] / lots_lis_factorized[0] ----
    first_lots_factorized = lis_lots_factorized[0]
    first_lots = lis_lots[0]

    num_lots = first_lots // first_lots_factorized

    return structure_name, num_lots
    

In [19]:
def unhedge_structure_into_outrights(lis_structure_names, lis_starting_contracts, lis_num_lots):
    # ---- Input validation ----
    if not (len(lis_structure_names) == len(lis_starting_contracts) == len(lis_num_lots)):
        raise ValueError("All input lists must have equal length.")

    # ---------- 1. Bulk fetch all structure patterns ----------
    conn = sqlite3.connect("positionmanager.db")
    cur = conn.cursor()

    placeholders = ",".join(["?"] * len(lis_structure_names))

    cur.execute(f"""
        SELECT structure_name, structure_lots 
        FROM custom_strategies
        WHERE structure_name IN ({placeholders})
    """, lis_structure_names)

    rows = cur.fetchall()
    conn.close()

    lots_lookup = {name: ast.literal_eval(lots_str) for name, lots_str in rows}

    # ---------- 2. Aggregate lots per contract ----------
    aggregated = {}   # ex: { "H25": 5, "M25": -3 }

    for structure_name, starting_contract, num_lots in zip(
        lis_structure_names, lis_starting_contracts, lis_num_lots
    ):
        if structure_name not in lots_lookup:
            raise ValueError(f"Strategy '{structure_name}' not found in custom_strategies.")

        lots_pattern = lots_lookup[structure_name]
        n = len(lots_pattern)

        for i in range(n):
            contract = next_contract(starting_contract, i)
            lots = lots_pattern[i] * num_lots

            aggregated[contract] = aggregated.get(contract, 0) + lots

    # ---------- 3. Convert dict → list of tuples ----------
    combined = list(aggregated.items())    # [(contract, lots), ...]

    # ---------- 4. Sort by proper month/year order ----------
    combined_sorted = sorted(combined, key=lambda x: contract_sort_key(x[0]))

    # ---------- 5. Unpack ----------
    final_contracts = [c for c, l in combined_sorted]
    final_lots = [l for c, l in combined_sorted]

    return final_contracts, final_lots

In [31]:
def implement_hedge(product, lis_structure_names, lis_starting_contracts, lis_num_lots, ):
    outright_contracts, outright_lots = unhedge_structure_into_outrights(lis_structure_names, lis_starting_contracts, lis_num_lots)
    hedged_structure_name, hedged_structure_lots = hedge_outrights(outright_contracts, outright_lots)
    hedged_structure_base_contract = outright_contracts[0]

    return hedged_structure_base_contract, hedged_structure_name, hedged_structure_lots

In [ ]:
def reflect_hedge_in_db

In [79]:
strategy_name, lis_lots = '1 mo dfly', [15,-45,45,-15]

create_custom_strategy_intraproduct(strategy_name, lis_lots)


In [51]:
product, lis_contracts, lis_lots = 'CL', ['H25', 'J25', 'K25'], [5,-10,5]
hedge_outrights(product, lis_contracts, lis_lots)

{'structure_name': '1 mo butterfly', 'lots': 5}

In [97]:
product, lis_contracts, lis_lots = 'CL', ['H25', 'J25', 'K25'], [5,-10,5]
unhedge_structure_into_outrights(['1 mo butterfly', '1 mo dfly'], ['G25', 'Z24'], [5, 10])

(['Z24', 'F25', 'G25', 'H25', 'J25'], [10, -30, 35, -20, 5])

In [25]:
conn = sqlite3.connect("positionmanager.db")
df = pd.read_sql_query("SELECT * FROM cl_positions", conn)
conn.close()

print(df)

   contract  lots  outright  1mo spread
0       Z25     3         3           3
1       F26    -5        -5           0
2       G26     3         3           0
3       H26    -5        -5           0
4       J26     3         3           0
5       K26    -5        -5           0
6       M26     3         3           0
7       N26    -5        -5           0
8       Q26     3         3           0
9       U26    -5        -5           0
10      V26     3         3           0
11      X26    -5        -5           0
12      Z26     3         3           0
13      F27    -5        -5           0
14      G27     3         3           0
15      H27    -5        -5           0
16      J27     3         3           0
17      K27    -5        -5           0
18      M27     3         3           0
19      N27    -5        -5           0
20      Q27     3         3           0
21      U27    -5        -5           0
22      V27     3         3           0
23      X27    -5        -5           0


In [30]:
conn = sqlite3.connect("positionmanager.db")
cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

conn.close()

print(tables)

[('custom_strategies',), ('cl_positions',), ('cl_structures',), ('positions_new',)]
